```
Copyright 2022 IBM Corporation

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

# Graph Features Extraction for Anti-Money Laudering

The Snap ML GraphFeaturePreprocessor is a scikit-learn compatible preprocessor that enables scalable and real-time feature extraction from graph-structured data. It provides utilities for creating and updating in-memory graphs as well as extracting new features from these graphs. The goal of this example is to show how to use the API of this preprocessor. As input, we will use a synthethic dataset in tabular format where each row represents a financial transaction. For each transaction 4 features are available: transaction ID, source account ID, target accound ID and transaction timestamp. 

In [1]:
# Import the Graph Feature Preprocessor from Snap ML
from snapml import GraphFeaturePreprocessor

# Import other libraries
import numpy as np
import time
import json
import pandas as pd
from IPython.display import display
import datetime

pd.options.display.max_columns = None

In [2]:
mon_laun_path = "../datasets/mon_laun_dataset/HI-Small_Trans.csv"
"""
0+10
01+0
"""
mon_laun_df = pd.read_csv(mon_laun_path)
mon_laun_df = mon_laun_df[mon_laun_df["Payment Format"] != "Reinvestment"]
mon_laun_df["transactionID"] = mon_laun_df.index.astype(float)
mon_laun_df["sourceAccount"] = (mon_laun_df["From Bank"].astype(str) + mon_laun_df["Account"])
mon_laun_df["sourceAccountID"] = pd.factorize(mon_laun_df['sourceAccount'].tolist())[0].astype(float)
mon_laun_df["targetAccount"] = mon_laun_df["To Bank"].astype(str) + mon_laun_df["Account.1"] 
mon_laun_df["targetAccountID"] = pd.factorize(mon_laun_df['targetAccount'].tolist())[0].astype(float)


mon_laun_df["Timestamp"] = pd.to_datetime(mon_laun_df["Timestamp"], format='%Y/%m/%d %H:%M')
mon_laun_df["timestamp"] = (mon_laun_df["Timestamp"].dt.hour*60+mon_laun_df["Timestamp"].dt.minute).astype(float)
mon_laun_df = mon_laun_df[mon_laun_df["transactionID"] < 5000.0]


display(mon_laun_df)

mon_laun_temp = mon_laun_df
mon_laun_df = mon_laun_df.drop(columns = ['Timestamp', "From Bank", "Account", "To Bank", "sourceAccount", "targetAccount", \
                            "Account.1","Amount Received", "Receiving Currency", "Amount Paid","Payment Currency", "Payment Format", "Is Laundering"])



mon_laun_arr = mon_laun_df.to_numpy()
display(mon_laun_arr)

/tmp/ipykernel_12633/1144248902.py:10: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  mon_laun_df["sourceAccountID"] = pd.factorize(mon_laun_df['sourceAccount'].tolist())[0].astype(float)
/tmp/ipykernel_12633/1144248902.py:12: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  mon_laun_df["targetAccountID"] = pd.factorize(mon_laun_df['targetAccount'].tolist())[0].astype(float)


,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,transactionID,sourceAccount,sourceAccountID,targetAccount,targetAccountID,timestamp
1,2022-09-01 00:20:00,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0,1.0,32088000F4580,0.0,18000F5340,0.0,20.0
8,2022-09-01 00:26:00,12,8000EC280,2439,8017BF800,7.66,US Dollar,7.66,US Dollar,Credit Card,0,8.0,128000EC280,1.0,24398017BF800,1.0,26.0
9,2022-09-01 00:21:00,1,8000EDEC0,211050,80AEF5310,383.71,US Dollar,383.71,US Dollar,Credit Card,0,9.0,18000EDEC0,2.0,21105080AEF5310,2.0,21.0
10,2022-09-01 00:04:00,1,8000F4510,11813,8011305D0,9.82,US Dollar,9.82,US Dollar,Credit Card,0,10.0,18000F4510,3.0,118138011305D0,3.0,4.0
12,2022-09-01 00:08:00,1,8000F4FE0,245335,812ED62E0,4.01,US Dollar,4.01,US Dollar,Credit Card,0,12.0,18000F4FE0,4.0,245335812ED62E0,4.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4971,2022-09-01 00:14:00,1,800327650,1688,800878710,28269.24,US Dollar,28269.24,US Dollar,Cheque,0,4971.0,1800327650,761.0,1688800878710,829.0,14.0
4972,2022-09-01 00:12:00,1,800327650,1688,800878710,6278.23,US Dollar,6278.23,US Dollar,Credit Card,0,4972.0,1800327650,761.0,1688800878710,829.0,12.0
4973,2022-09-01 00:15:00,1,800327650,1688,800878710,26820.77,US Dollar,26820.77,US Dollar,Cash,0,4973.0,1800327650,761.0,1688800878710,829.0,15.0
4980,2022-09-01 00:08:00,11157,800879820,35925,80E1EEE10,140.28,US Dollar,140.28,US Dollar,Credit Card,0,4980.0,11157800879820,762.0,3592580E1EEE10,830.0,8.0


array([[1.000e+00, 0.000e+00, 0.000e+00, 2.000e+01],
       [8.000e+00, 1.000e+00, 1.000e+00, 2.600e+01],
       [9.000e+00, 2.000e+00, 2.000e+00, 2.100e+01],
       ...,
       [4.973e+03, 7.610e+02, 8.290e+02, 1.500e+01],
       [4.980e+03, 7.620e+02, 8.300e+02, 8.000e+00],
       [4.985e+03, 7.630e+02, 8.310e+02, 5.000e+00]])

Here we assume that the user has access to a set of (labeled) transactions with raw features which could be used to train a machine learning (ML) model, e.g., for fraud detection. The user will extract graph features using the Graph Features Preprocessor which will be added to the initial raw features present in the transactions. The enriched set of features will be used to train an ML model. The main steps associated with this use case are shown below:

<div> <img src="img/gfp-use-case1.png" width="1000"> </div>


In [3]:
# Path to the file that contains financial transactions, e.g., used for training ML models
train_graph_path = "../datasets/graph_feature_preprocessor/aml_custom_train.txt"

print("Loading the transactions ")
X_train = np.loadtxt(train_graph_path, dtype=np.float64, delimiter=" ", comments="#", usecols=range(4))
X_train = mon_laun_arr
print("Input dataset shape: ", X_train.shape)

df = pd.DataFrame(X_train, columns=['transactionID', 'sourceAccountID', 'targetAccountID', 'timestamp'])
display(df)
df.dtypes

Loading the transactions 
Input dataset shape:  (1397, 4)


,transactionID,sourceAccountID,targetAccountID,timestamp
0,1.0,0.0,0.0,20.0
1,8.0,1.0,1.0,26.0
2,9.0,2.0,2.0,21.0
3,10.0,3.0,3.0,4.0
4,12.0,4.0,4.0,8.0
...,...,...,...,...
1392,4971.0,761.0,829.0,14.0
1393,4972.0,761.0,829.0,12.0
1394,4973.0,761.0,829.0,15.0
1395,4980.0,762.0,830.0,8.0


transactionID      float64
sourceAccountID    float64
targetAccountID    float64
timestamp          float64
dtype: object

In [4]:
# The following dictionary defines the configuration parameters of the Graph Feature Preprocessor

params = {
    "num_threads": 4,             # number of software threads to be used (important for performance)
    "time_window": 16,            # time window used if no pattern was specified
    
    "vertex_stats": True,         # produce vertex statistics
    "vertex_stats_cols": [3],     # produce vertex statistics using the selected input columns
    
    # features: 0:fan,1:deg,2:ratio,3:avg,4:sum,5:min,6:max,7:median,8:var,9:skew,10:kurtosis
    "vertex_stats_feats": [0, 1, 2, 3, 4, 8, 9, 10],  # fan,deg,ratio,avg,sum,var,skew,kurtosis
    
    # fan in/out parameters
    "fan": True,
    "fan_tw": 16,
    "fan_bins": [y+2 for y in range(2)],
    
    # in/out degree parameters
    "degree": True,
    "degree_tw": 16,
    "degree_bins": [y+2 for y in range(2)],
    
    # scatter gather parameters
    "scatter-gather": True,
    "scatter-gather_tw": 16,
    "scatter-gather_bins": [y+2 for y in range(2)],
    
    # temporal cycle parameters
    "temp-cycle": True,
    "temp-cycle_tw": 16,
    "temp-cycle_bins": [y+2 for y in range(2)],
    
    # length-constrained simple cycle parameters
    "lc-cycle": False,
    "lc-cycle_tw": 16,
    "lc-cycle_len": 8,
    "lc-cycle_bins": [y+2 for y in range(2)],
}

In [5]:
# Create a Graph Feature Preprocessor, set its configuration using the above dictionary and verify it

print("Creating a graph feature preprocessor ")
gp = GraphFeaturePreprocessor()

print("Setting the parameters of the graph feature preprocessor ")
gp.set_params(params)

print("Graph feature preprocessor parameters: ", json.dumps(gp.get_params(), indent=4))

Creating a graph feature preprocessor 
Setting the parameters of the graph feature preprocessor 
Graph feature preprocessor parameters:  {
    "num_threads": 4,
    "time_window": 16,
    "max_no_edges": -1,
    "vertex_stats": true,
    "vertex_stats_tw": 1728000,
    "vertex_stats_cols": [
        3
    ],
    "vertex_stats_feats": [
        0,
        1,
        2,
        3,
        4,
        8,
        9,
        10
    ],
    "fan": true,
    "fan_tw": 16,
    "fan_bins": [
        2,
        3
    ],
    "degree": true,
    "degree_tw": 16,
    "degree_bins": [
        2,
        3
    ],
    "scatter-gather": true,
    "scatter-gather_tw": 16,
    "scatter-gather_bins": [
        2,
        3
    ],
    "temp-cycle": true,
    "temp-cycle_tw": 16,
    "temp-cycle_bins": [
        2,
        3
    ],
    "lc-cycle": false,
    "lc-cycle_tw": 16,
    "lc-cycle_len": 8,
    "lc-cycle_bins": [
        2,
        3
    ]
}


In [6]:
print("Enriching the transactions with new graph features ")
print("Raw dataset shape: ", X_train.shape)

# the fit_transform and transform functions are equivalent
# these functions can run on single transactions or on batches of transactions
X_train_enriched = gp.fit_transform(X_train.astype("float64")) 

print("Enriched dataset shape: ", X_train_enriched.shape)

Enriching the transactions with new graph features 
Raw dataset shape:  (1397, 4)
Enriched dataset shape:  (1397, 48)


We define a helper function to inspect the newly generated graph-based features for a given transaction:

In [7]:
def print_enriched_transaction(transaction, params):
    colnames = []
    
    # add raw features names
    colnames.append("transactionID")
    colnames.append("sourceAccountID")
    colnames.append("targetAccountID")
    colnames.append("timestamp")
    
    # add features names for the graph patterns
    for pattern in ['fan', 'degree', 'scatter-gather', 'temp-cycle', 'lc-cycle']:
        if pattern in params:
            if params[pattern]:
                bins = len(params[pattern +'_bins'])
                if pattern in ['fan', 'degree']:
                    for i in range(bins-1):
                        colnames.append(pattern+"_in_bins_"+str(params[pattern +'_bins'][i])+"-"+str(params[pattern +'_bins'][i+1]))
                    colnames.append(pattern+"_in_bins_"+str(params[pattern +'_bins'][i+1])+"-inf")
                    for i in range(bins-1):
                        colnames.append(pattern+"_out_bins_"+str(params[pattern +'_bins'][i])+"-"+str(params[pattern +'_bins'][i+1]))
                    colnames.append(pattern+"_out_bins_"+str(params[pattern +'_bins'][i+1])+"-inf")
                else:
                    for i in range(bins-1):
                        colnames.append(pattern+"_bins_"+str(params[pattern +'_bins'][i])+"-"+str(params[pattern +'_bins'][i+1]))
                    colnames.append(pattern+"_bins_"+str(params[pattern +'_bins'][i+1])+"-inf")

    vert_feat_names = ["fan","deg","ratio","avg","sum","min","max","median","var","skew","kurtosis"]

    # add features names for the vertex statistics
    for orig in ['source', 'dest']:
        for direction in ['out', 'in']:
            # add fan, deg, and ratio features
            for k in [0, 1, 2]:
                if k in params["vertex_stats_feats"]:
                    feat_name = orig + "_" + vert_feat_names[k] + "_" + direction
                    colnames.append(feat_name)
            for col in params["vertex_stats_cols"]:
                # add avg, sum, min, max, median, var, skew, and kurtosis features
                for k in [3, 4, 5, 6, 7, 8, 9, 10]:
                    if k in params["vertex_stats_feats"]:
                        feat_name = orig + "_" + vert_feat_names[k] + "_col" + str(col) + "_" + direction
                        colnames.append(feat_name)

    df = pd.DataFrame(transaction, columns=colnames)
    return df

In [8]:
print("Enriched transactions: ")
df_enriched = print_enriched_transaction(X_train_enriched, gp.get_params())
mon_laun_temp = mon_laun_temp.drop(columns = ['sourceAccountID', 'targetAccountID', 'timestamp'])


display(df_enriched)


Enriched transactions: 


,transactionID,sourceAccountID,targetAccountID,timestamp,fan_in_bins_2-3,fan_in_bins_3-inf,fan_out_bins_2-3,fan_out_bins_3-inf,degree_in_bins_2-3,degree_in_bins_3-inf,degree_out_bins_2-3,degree_out_bins_3-inf,scatter-gather_bins_2-3,scatter-gather_bins_3-inf,temp-cycle_bins_2-3,temp-cycle_bins_3-inf,source_fan_out,source_deg_out,source_ratio_out,source_avg_col3_out,source_sum_col3_out,source_var_col3_out,source_skew_col3_out,source_kurtosis_col3_out,source_fan_in,source_deg_in,source_ratio_in,source_avg_col3_in,source_sum_col3_in,source_var_col3_in,source_skew_col3_in,source_kurtosis_col3_in,dest_fan_out,dest_deg_out,dest_ratio_out,dest_avg_col3_out,dest_sum_col3_out,dest_var_col3_out,dest_skew_col3_out,dest_kurtosis_col3_out,dest_fan_in,dest_deg_in,dest_ratio_in,dest_avg_col3_in,dest_sum_col3_in,dest_var_col3_in,dest_skew_col3_in,dest_kurtosis_col3_in
0,1.0,8.0,9.0,10.0,0.0,6.0,0.0,3.0,0.0,6.0,0.0,3.0,5.0,11.0,0.0,3.0,11.0,17.0,1.545455,10.000000,250.0,0.000000,0.000000,0.000000,11.0,15.0,1.363636,9.000000,252.0,0.000000,0.000000,0.000000,8.0,8.0,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,10.0,17.0,1.700000,15.473684,588.0,57.617729,0.66564,1.443077
1,12.0,13.0,14.0,22.0,0.0,10.0,0.0,5.0,0.0,10.0,0.0,5.0,8.0,7.0,0.0,10.0,14.0,18.0,1.285714,17.659091,777.0,63.042872,0.032858,1.402103,11.0,15.0,1.363636,19.714286,414.0,53.061224,0.490290,1.240385,11.0,14.0,1.272727,0.000000,0.0,0.000000,0.000000,0.000000,10.0,14.0,1.400000,22.000000,374.0,0.000000,0.00000,0.000000
2,24.0,25.0,28.0,29.0,0.0,8.0,0.0,5.0,0.0,8.0,0.0,5.0,8.0,5.0,0.0,4.0,7.0,8.0,1.142857,29.000000,348.0,0.000000,0.000000,0.000000,12.0,14.0,1.166667,0.000000,0.0,0.000000,0.000000,0.000000,10.0,14.0,1.400000,0.000000,0.0,0.000000,0.000000,0.000000,8.0,9.0,1.125000,29.000000,348.0,0.000000,0.00000,0.000000
3,32.0,33.0,34.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.000000,38.000000,38.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.0,2.0,2.000000,36.000000,72.0,0.000000,0.000000,0.000000,1.0,1.0,1.000000,38.000000,38.0,0.000000,0.00000,0.000000
4,39.0,40.0,42.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.000000,43.000000,43.0,0.000000,0.000000,0.000000,1.0,2.0,2.000000,41.000000,82.0,0.000000,0.000000,0.000000,1.0,2.0,2.000000,44.000000,88.0,0.000000,0.000000,0.000000,1.0,1.0,1.000000,43.000000,43.0,0.000000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1392,8.0,29.0,19.0,15.0,1.0,12.0,0.0,5.0,0.0,13.0,0.0,5.0,6.0,5.0,0.0,10.0,14.0,16.0,1.142857,29.000000,377.0,0.000000,0.000000,0.000000,9.0,15.0,1.666667,24.804878,1017.0,18.596074,-0.753492,1.950026,9.0,11.0,1.222222,20.000000,300.0,0.000000,0.000000,0.000000,9.0,12.0,1.333333,0.000000,0.0,0.000000,0.00000,0.000000
1393,13.0,13.0,13.0,19.0,0.0,8.0,0.0,3.0,0.0,8.0,0.0,3.0,2.0,5.0,0.0,15.0,14.0,18.0,1.285714,17.659091,777.0,63.042872,0.032858,1.402103,11.0,15.0,1.363636,19.714286,414.0,53.061224,0.490290,1.240385,14.0,18.0,1.285714,17.659091,777.0,63.042872,0.032858,1.402103,11.0,15.0,1.363636,19.714286,414.0,53.061224,0.49029,1.240385
1394,8.0,2.0,22.0,6.0,1.0,12.0,0.0,5.0,0.0,13.0,0.0,5.0,6.0,5.0,0.0,10.0,7.0,9.0,1.285714,0.000000,0.0,0.000000,0.000000,0.000000,12.0,14.0,1.166667,27.000000,405.0,0.000000,0.000000,0.000000,13.0,20.0,1.538462,24.000000,336.0,0.000000,0.000000,0.000000,10.0,13.0,1.300000,0.000000,0.0,0.000000,0.00000,0.000000
1395,28.0,8.0,17.0,14.0,0.0,3.0,0.0,3.0,2.0,3.0,2.0,3.0,2.0,3.0,0.0,3.0,11.0,17.0,1.545455,10.000000,250.0,0.000000,0.000000,0.000000,11.0,15.0,1.363636,9.000000,252.0,0.000000,0.000000,0.000000,6.0,7.0,1.166667,0.000000,0.0,0.000000,0.000000,0.000000,14.0,21.0,1.500000,18.000000,144.0,0.000000,0.00000,0.000000


In [14]:
display(mon_laun_temp)

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,transactionID,sourceAccount,targetAccount
1,2022-09-01 00:20:00,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0,1.0,32088000F4580,18000F5340
8,2022-09-01 00:26:00,12,8000EC280,2439,8017BF800,7.66,US Dollar,7.66,US Dollar,Credit Card,0,8.0,128000EC280,24398017BF800
9,2022-09-01 00:21:00,1,8000EDEC0,211050,80AEF5310,383.71,US Dollar,383.71,US Dollar,Credit Card,0,9.0,18000EDEC0,21105080AEF5310
10,2022-09-01 00:04:00,1,8000F4510,11813,8011305D0,9.82,US Dollar,9.82,US Dollar,Credit Card,0,10.0,18000F4510,118138011305D0
12,2022-09-01 00:08:00,1,8000F4FE0,245335,812ED62E0,4.01,US Dollar,4.01,US Dollar,Credit Card,0,12.0,18000F4FE0,245335812ED62E0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4971,2022-09-01 00:14:00,1,800327650,1688,800878710,28269.24,US Dollar,28269.24,US Dollar,Cheque,0,4971.0,1800327650,1688800878710
4972,2022-09-01 00:12:00,1,800327650,1688,800878710,6278.23,US Dollar,6278.23,US Dollar,Credit Card,0,4972.0,1800327650,1688800878710
4973,2022-09-01 00:15:00,1,800327650,1688,800878710,26820.77,US Dollar,26820.77,US Dollar,Cash,0,4973.0,1800327650,1688800878710
4980,2022-09-01 00:08:00,11157,800879820,35925,80E1EEE10,140.28,US Dollar,140.28,US Dollar,Credit Card,0,4980.0,11157800879820,3592580E1EEE10


This newly enriched set of transactions can now be used to train a ML model. Once trained, the model can be used for prediction (e.g., detect anomalies) on new (unlabeled) transactions. The main steps associated with this use case is shown below:

<div> <img src="img/gfp-use-case2.png" width="1000"> </div>

In [20]:

df_enriched['transactionID'].nunique()

849

In [16]:

df_xgboost = pd.merge(df_enriched, mon_laun_temp, on='transactionID')
display(df_xgboost)

,transactionID,sourceAccountID,targetAccountID,timestamp,fan_in_bins_2-3,fan_in_bins_3-inf,fan_out_bins_2-3,fan_out_bins_3-inf,degree_in_bins_2-3,degree_in_bins_3-inf,degree_out_bins_2-3,degree_out_bins_3-inf,scatter-gather_bins_2-3,scatter-gather_bins_3-inf,temp-cycle_bins_2-3,temp-cycle_bins_3-inf,source_fan_out,source_deg_out,source_ratio_out,source_avg_col3_out,source_sum_col3_out,source_var_col3_out,source_skew_col3_out,source_kurtosis_col3_out,source_fan_in,source_deg_in,source_ratio_in,source_avg_col3_in,source_sum_col3_in,source_var_col3_in,source_skew_col3_in,source_kurtosis_col3_in,dest_fan_out,dest_deg_out,dest_ratio_out,dest_avg_col3_out,dest_sum_col3_out,dest_var_col3_out,dest_skew_col3_out,dest_kurtosis_col3_out,dest_fan_in,dest_deg_in,dest_ratio_in,dest_avg_col3_in,dest_sum_col3_in,dest_var_col3_in,dest_skew_col3_in,dest_kurtosis_col3_in,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,sourceAccount,targetAccount
0,1.0,8.0,9.0,10.0,0.0,6.0,0.0,3.0,0.0,6.0,0.0,3.0,5.0,11.0,0.0,3.0,11.0,17.0,1.545455,10.000000,250.0,0.000000,0.000000,0.000000,11.0,15.0,1.363636,9.000000,252.0,0.000000,0.000000,0.000000,8.0,8.0,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,10.0,17.0,1.700000,15.473684,588.0,57.617729,0.66564,1.443077,2022-09-01 00:20:00,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0,32088000F4580,18000F5340
1,12.0,13.0,14.0,22.0,0.0,10.0,0.0,5.0,0.0,10.0,0.0,5.0,8.0,7.0,0.0,10.0,14.0,18.0,1.285714,17.659091,777.0,63.042872,0.032858,1.402103,11.0,15.0,1.363636,19.714286,414.0,53.061224,0.490290,1.240385,11.0,14.0,1.272727,0.000000,0.0,0.000000,0.000000,0.000000,10.0,14.0,1.400000,22.000000,374.0,0.000000,0.00000,0.000000,2022-09-01 00:08:00,1,8000F4FE0,245335,812ED62E0,4.01,US Dollar,4.01,US Dollar,Credit Card,0,18000F4FE0,245335812ED62E0
2,24.0,25.0,28.0,29.0,0.0,8.0,0.0,5.0,0.0,8.0,0.0,5.0,8.0,5.0,0.0,4.0,7.0,8.0,1.142857,29.000000,348.0,0.000000,0.000000,0.000000,12.0,14.0,1.166667,0.000000,0.0,0.000000,0.000000,0.000000,10.0,14.0,1.400000,0.000000,0.0,0.000000,0.000000,0.000000,8.0,9.0,1.125000,29.000000,348.0,0.000000,0.00000,0.000000,2022-09-01 00:21:00,10,8005F0B50,13327,810B0FB40,86.30,US Dollar,86.30,US Dollar,Credit Card,0,108005F0B50,13327810B0FB40
3,32.0,33.0,34.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.000000,38.000000,38.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.0,2.0,2.000000,36.000000,72.0,0.000000,0.000000,0.000000,1.0,1.0,1.000000,38.000000,38.0,0.000000,0.00000,0.000000,2022-09-01 00:26:00,10,800058100,1688,8005FD430,159271.00,US Dollar,159271.00,US Dollar,ACH,0,10800058100,16888005FD430
4,39.0,40.0,42.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.000000,43.000000,43.0,0.000000,0.000000,0.000000,1.0,2.0,2.000000,41.000000,82.0,0.000000,0.000000,0.000000,1.0,2.0,2.000000,44.000000,88.0,0.000000,0.000000,0.000000,1.0,1.0,1.000000,43.000000,43.0,0.000000,0.00000,0.000000,2022-09-01 00:15:00,1674,800AC8E30,1420,801008520,5069.19,US Dollar,5069.19,US Dollar,Cheque,0,1674800AC8E30,1420801008520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,8.0,29.0,19.0,15.0,1.0,12.0,0.0,5.0,0.0,13.0,0.0,5.0,6.0,5.0,0.0,10.0,14.0,16.0,1.142857,29.000000,377.0,0.000000,0.000000,0.000000,9.0,15.0,1.666667,24.804878,1017.0,18.596074,-0.753492,1.950026,9.0,11.0,1.222222,20.000000,300.0,0.000000,0.000000,0.000000,9.0,12.0,1.333333,0.000000,0.0,0.000000,0.00000,0.000000,2022-09-01 00:26:00,12,8000EC280,2439,8017BF800,7.66,US Dollar,7.66,US Dollar,Credit Card,0,128000EC280,24398017BF800
682,13.0,13.0,13.0,19.0,0.0,8.0,0.0,3.0,0.0,8.0,0.0,3.0,2.0,5.0,0.0,15.0,14.0,18.0,1.285714,17.659091,777.0,63.042872,0.032858,1.402103,11.0,15.

In [21]:
df_xgboost.to_csv('../datasets/mon_laun_dataset/merged_dataset.csv')

In [11]:
# Path to the file that contains financial transactions used for testing
test_transactions_path = "../datasets/graph_feature_preprocessor/aml_custom_test.txt"

print("Loading the test transactions ")
X_test = np.loadtxt(test_transactions_path, dtype=np.float64, delimiter=" ", comments="#", usecols=range(4))
print("Input dataset shape: ", X_test.shape)

df = pd.DataFrame(X_test, columns=['transactionID', 'sourceAccountID', 'destinationAccountID', 'timestamp'])
display(df)

Loading the test transactions 
Input dataset shape:  (8, 4)


,transactionID,sourceAccountID,destinationAccountID,timestamp
0,8.0,8.0,9.0,8.0
1,9.0,9.0,10.0,9.0
2,10.0,10.0,11.0,10.0
3,11.0,9.0,11.0,11.0
4,12.0,11.0,9.0,12.0
5,13.0,11.0,8.0,13.0
6,14.0,8.0,10.0,14.0
7,15.0,10.0,8.0,15.0


In [12]:
print("Creating a graph feature preprocessor ")
gp = GraphFeaturePreprocessor()

print("Setting the parameters of the graph feature preprocessor ")
gp.set_params(params)

print("Creating the graph using the training transactions ")
gp.fit(X_train)  # this step is optional, however recommended for capturing deeper graph feature

# transform can run on single transactions or on batches of transactions
print("Enriching the test transactions with new graph features ")
X_test_enriched = gp.transform(X_test.astype("float64"))
print_enriched_transaction(X_test_enriched, gp.get_params())

Creating a graph feature preprocessor 
Setting the parameters of the graph feature preprocessor 
Creating the graph using the training transactions 
Enriching the test transactions with new graph features 


,transactionID,sourceAccountID,targetAccountID,timestamp,fan_in_bins_2-3,fan_in_bins_3-inf,fan_out_bins_2-3,fan_out_bins_3-inf,degree_in_bins_2-3,degree_in_bins_3-inf,degree_out_bins_2-3,degree_out_bins_3-inf,scatter-gather_bins_2-3,scatter-gather_bins_3-inf,temp-cycle_bins_2-3,temp-cycle_bins_3-inf,source_fan_out,source_deg_out,source_ratio_out,source_avg_col3_out,source_sum_col3_out,source_var_col3_out,source_skew_col3_out,source_kurtosis_col3_out,source_fan_in,source_deg_in,source_ratio_in,source_avg_col3_in,source_sum_col3_in,source_var_col3_in,source_skew_col3_in,source_kurtosis_col3_in,dest_fan_out,dest_deg_out,dest_ratio_out,dest_avg_col3_out,dest_sum_col3_out,dest_var_col3_out,dest_skew_col3_out,dest_kurtosis_col3_out,dest_fan_in,dest_deg_in,dest_ratio_in,dest_avg_col3_in,dest_sum_col3_in,dest_var_col3_in,dest_skew_col3_in,dest_kurtosis_col3_in
0,8.0,8.0,9.0,8.0,0.0,4.0,0.0,2.0,0.0,4.0,0.0,2.0,0.0,1.0,0.0,5.0,12.0,17.0,1.416667,10.000000,230.0,0.000000,0.000000,0.00000,11.0,15.0,1.363636,9.000000,234.0,0.00000,0.00000,0.000000,9.0,10.0,1.111111,0.000000,0.0,0.000000,0.000000,0.00000,11.0,16.0,1.454545,15.675676,580.0,57.84076,0.61756,1.387503
1,9.0,9.0,10.0,9.0,0.0,2.0,0.0,3.0,0.0,2.0,0.0,3.0,3.0,8.0,0.0,25.0,9.0,10.0,1.111111,0.000000,0.0,0.000000,0.000000,0.00000,11.0,16.0,1.454545,15.675676,580.0,57.84076,0.61756,1.387503,11.0,18.0,1.636364,14.000000,280.0,0.000000,0.000000,0.00000,9.0,11.0,1.222222,0.000000,0.0,0.00000,0.00000,0.000000
2,10.0,10.0,11.0,10.0,0.0,5.0,0.0,5.0,0.0,5.0,0.0,5.0,4.0,7.0,0.0,21.0,11.0,18.0,1.636364,14.000000,280.0,0.000000,0.000000,0.00000,9.0,11.0,1.222222,0.000000,0.0,0.00000,0.00000,0.000000,15.0,18.0,1.200000,19.860465,854.0,32.213088,-0.204796,1.34014,11.0,14.0,1.272727,0.000000,0.0,0.00000,0.00000,0.000000
3,11.0,9.0,11.0,11.0,0.0,1.0,0.0,3.0,0.0,1.0,0.0,3.0,1.0,1.0,1.0,39.0,9.0,10.0,1.111111,0.000000,0.0,0.000000,0.000000,0.00000,11.0,16.0,1.454545,15.675676,580.0,57.84076,0.61756,1.387503,15.0,18.0,1.200000,19.860465,854.0,32.213088,-0.204796,1.34014,11.0,14.0,1.272727,0.000000,0.0,0.00000,0.00000,0.000000
4,12.0,11.0,9.0,12.0,0.0,4.0,1.0,2.0,0.0,4.0,1.0,2.0,1.0,3.0,1.0,6.0,15.0,18.0,1.200000,19.860465,854.0,32.213088,-0.204796,1.34014,11.0,14.0,1.272727,0.000000,0.0,0.00000,0.00000,0.000000,9.0,10.0,1.111111,0.000000,0.0,0.000000,0.000000,0.00000,11.0,16.0,1.454545,15.675676,580.0,57.84076,0.61756,1.387503
5,13.0,11.0,8.0,13.0,0.0,4.0,0.0,2.0,0.0,4.0,0.0,2.0,0.0,1.0,0.0,21.0,15.0,18.0,1.200000,19.860465,854.0,32.213088,-0.204796,1.34014,11.0,14.0,1.272727,0.000000,0.0,0.00000,0.00000,0.000000,12.0,17.0,1.416667,10.000000,230.0,0.000000,0.000000,0.00000,11.0,15.0,1.363636,9.000000,234.0,0.00000,0.00000,0.000000
6,14.0,8.0,10.0,14.0,0.0,3.0,0.0,1.0,0.0,3.0,0.0,1.0,2.0,2.0,0.0,9.0,12.0,17.0,1.416667,10.000000,230.0,0.000000,0.000000,0.00000,11.0,15.0,1.363636,9.000000,234.0,0.00000,0.00000,0.000000,11.0,18.0,1.636364,14.000000,280.0,0.000000,0.000000,0.00000,9.0,11.0,1.222222,0.000000,0.0,0.00000,0.00000,0.000000
7,15.0,10.0,8.0,15.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,2.0,3.0,0.0,4.0,11.0,18.0,1.636364,14.000000,280.0,0.000000,0.000000,0.00000,9.0,11.0,1.222222,0.000000,0.0,0.00000,0.00000,0.000000,12.0,17.0,1.416667,10.000000,230.0,0.000000,0.000000,0.00000,11.0,15.0,1.363636,9.000000,234.0,0.00000,0.00000,0.000000


Now the enriched transactions can be used as input to the ML model previously trained. 